# Campus Recruitment with SVM and grid Search
Author: Tino Merl

__Table of Contents:__
* [Motivation & Goal](#motiv_goal)
* [CRISP-DM](#crisp_dm)
* [Business Understanding](#buss_und)
* [Data Unterstanding](#dat_und)
* [Data Preparation](#dat_prep)
* [Modeling](#model)
* [Evaluation](#eval)

## Motivation & Goal<a class="anchor" id="motiv_goal"></a>
First and foremost this is a homework to deepen the understanding and usage of python's library scikit-learn. To be precise the usage of support vector machines(SVM) in scikit-learn. For this assignment the campus recruiment dataset from kaggle is used.[[1]](#kaggle_dataset) The goal is to use SVMs to answer the following questions tasked for the dataset.

1. Which factor influenced a candidate in getting placed?
2. Does percentage matters for one to get placed?
3. Which degree specialization is much demanded by corporate?
4. Play with the data conducting all statistical tests.

## CRISP-DM<a class="anchor" id="crisp_dm"></a>
CRISP-DM is an acronym standing for Cross Industry Standard Process for Data Mining. I will continue to refer to it as CRISP-DM. It's a widely used process for standardization while working with data. It consists of six steps.

1. Business Understanding
2. Data Unterstanding
3. Data Preparation
4. Modeling
5. Evaluation
6. Deployment

For the sake of this homework we will be leaving out the last step since this model will not be deployed to production. Therefore it will remain as experimental and we end the process after the fifth step.

## Business Understanding

Since there is no real business use case we will translate the assignment and the questions listed in the first paragraph.

## Footnotes
[[1]]<a class="anchor" id="kaggle_dataset"></a> Ben Roshan D (2020). Campus Recruitment, Academic and Employability Factors influencing placement, Version 1. Retrieved 2020-05-10 from https://www.kaggle.com/benroshan/factors-affecting-campus-placement.